<a href="https://colab.research.google.com/github/CodeofRahul/Fine_Tuning_Gemma_Model/blob/main/Fine_Tuning_Gemma_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- **transformers:** A library by Hugging Face for state-of-the-art natural language processing (NLP) models.
- **datasets:** To load and process datasets.
- **bitsandbytes:** Helps with memory-efficient quantization of large models.
- **accelerate:** For distributed training and faster model inference.
- **peft:** For Parameter-Efficient Fine-Tuning (PEFT) using methods like LoRA (Low-Rank Adaptation).
- **trl:** Stands for "Transformers Reinforcement Learning," useful for tasks involving RL in NLP.
- **py7zr:** A library for handling .7z archive files.

In [1]:
# Install necessary libraries
!pip install transformers datasets bitsandbytes accelerate peft trl py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363

- **torch:** PyTorch library for deep learning tasks.
- **load_dataset:** Loads datasets, here used for fetching the samsum dataset.
- **AutoTokenizer & AutoModelForCausalLM:** These load the pre-trained tokenizer and model for causal language modeling.
- **BitsAndBytesConfig:** Configures the quantization (memory-efficient) aspects of the model.
- **prepare_model_for_kbit_training & get_peft_model:** Helps prepare the model for k-bit fine-tuning and apply LoRA configurations for efficient training.
- **TrainingArguments & Trainer:** Used to define training parameters and handle the training loop.

In [2]:
# Import libraries
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer

- **load_dataset("samsum"):** Downloads and loads the dataset from the Hugging Face repository.

In [3]:
# Load the dataset
dataset = load_dataset('samsum')
print(f"Dataset loaded: {dataset}")

README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset loaded: DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})


## **Authentication**

In [5]:
from huggingface_hub import login
login()

Token has not been saved to git credential helper.


- **tokenizer:** Converts text into a format suitable for the model. Here, it's loaded from the pretrained "Gemma-2B" model.
- **tokenizer.pad_token:** Sets the padding token to be the end-of-sequence (EOS) token.

In [6]:

## Load the Gemma tokenizer and model
model_name = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

- **load_in_4bit=True:** Loads the model in a more memory-efficient 4-bit format.
- **bnb_4bit_use_double_quant=True:** Uses double quantization to further optimize memory usage.
- **bnb_4bit_quant_type="nf4":** Uses a specific type of quantization for higher accuracy (NF4 type).
- **bnb_4bit_compute_dtype=torch.bfloat16:** Sets the computation to use BFloat16 for efficient mixed precision.

In [7]:

# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

- **device_map="auto":** Automatically maps the model layers to available devices (CPU/GPU).

In [8]:
## Prepare the model for k-bit training
model = prepare_model_for_kbit_training(model)

- **r=16:** Sets the rank of the low-rank matrix in LoRA.
- **lora_alpha=32:** A scaling factor for LoRA.
- **lora_dropout=0.05:** Sets the dropout rate during training.
- **task_type="CAUSAL_LM":** Specifies that the task is causal language modeling (predicting the next word).

In [9]:

# Define LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


- **get_peft_model:** Wraps the original model with the LoRA parameters for efficient training.

In [10]:

# Get the PEFT model
model = get_peft_model(model, peft_config)

- **tokenize_function:** Tokenizes each dialogue and its corresponding summary.
- **labels["input_ids"]:** Replaces the padding tokens with -100 so they are ignored during loss computation.
- **dataset.map:** Applies the tokenization function to the entire dataset.

In [11]:
# Tokenize the dataset
def tokenize_function(examples):
    prompt = "Summarize the following conversation:\n\n"
    inputs = [prompt + dialog for dialog in examples["dialogue"]]
    targets = [summary for summary in examples["summary"]]

    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)
    labels = tokenizer(targets, max_length=512, padding="max_length", truncation=True)

    # Replace padding token IDs with -100 to ignore them in the loss computation
    labels["input_ids"] = [[-100 if token == tokenizer.pad_token_id else token for token in label] for label in labels["input_ids"]]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

- **output_dir:** Directory to save the results.
- **num_train_epochs=3:** Train for 3 epochs.
- **batch sizes:** Set the batch size for training and evaluation.
- **warmup_steps:** Number of warm-up steps during training.
- **evaluation_strategy="steps":** Evaluates the model periodically.
- **gradient_accumulation_steps:** Accumulates gradients over multiple steps for memory efficiency.

In [12]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    gradient_accumulation_steps=8,
#     fp16=True,  # Enable mixed precision training
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)


In [ ]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 420kumarahul (420kumarahul-education) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


In [ ]:
# Evaluate the model
evaluation = trainer.evaluate()
print(f"Evaluation results: {evaluation}")

In [ ]:
# Test the fine-tuned model
test_input = "Summarize the following conversation:\n\nJohn: Hey, how's it going?\nSarah: Not bad, just finished a big project at work.\nJohn: That's great! Want to grab dinner to celebrate?\nSarah: Sure, that sounds fun. Where should we go?\nJohn: How about that new Italian place downtown?\nSarah: Perfect, I've been wanting to try it. See you at 7?"

inputs = tokenizer(test_input, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
print("Generated summary:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_gemma_qlora")
tokenizer.save_pretrained("./fine_tuned_gemma_qlora")